In [1]:
import nbimporter
import numpy as np
import numpy.random as alea
import random as rd
import matplotlib.pyplot as plt
from math import *
import progressbar
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import shuffle
from procedure_MOM import procedure_MOM as PM

Importing Jupyter notebook from procedure_MOM.ipynb


In [8]:
class MOM_LASSO():
    
    def __init__(self , K): 
        
        self.K = K
        self.iter_max = 200
    
    def fit(self , X , Y , lamb = 1 , method = "ADMM" , initialize = "zero"):
        
        n,p = np.shape(X)
        j = n // self.K
        self.lamb = lamb
        
        if initialize == "zero":
            t = np.matrix(np.zeros((p,1)))
        
        if method == "ADMM" : 
                
            z = np.matrix(np.zeros((p,1))) 
            u = np.matrix(np.zeros((p,1))) 
            rho = 5 * np.identity(p)
            
            for l in range(self.iter_max) :
                
                k = PM.MOM_rand(PM.P_quadra(X , Y , t) , self.K)[1]
                Xk = X[k]
                Yk = Y[k]
                
                t = np.linalg.solve((Xk.T) * Xk  +  rhoM, (Xk.T) * Yk  +  rho * z  -  u)
                z = PM.soft_thresholding(self.lamb / rho,t + u / rho)
                u = u + rho * (t - z)
                
            self.t = t
            print("Training finished")
                
        if method == "ISTA" :
            
            for l in range(self.iter_max) :
                
                k = PM.MOM_rand(PM.P_quadra( X , Y , t ) , self.K)[1] 
                Xk = X[k]
                Yk = Y[k]
                
                #Beginning of backtracking with c = 1/2
                gamma = 1
                t_prev = t
                F = PM.quadra_loss(Xk , Yk , t_prev)
                
                t = PM.soft_thresholding(self.lamb * gamma,t - gamma * PM.grad(Xk , Yk , t))
                delta = PM.quadra_loss(Xk , Yk , t) - F - PM.grad(Xk , Yk , t_prev).T * (t - t_prev) - (1 / (2 * gamma)) * np.linalg.norm(t - t_prev) ** 2
                
                while delta > 1e-3 :
                    
                    gamma *= mu
                    t = PM.soft_thresholding(self.lamb * gamma , t_prev - gamma * PM.grad(Xk , Yk , t_prev))
                    delta = PM.quadra_loss(Xk , Yk , t) - F - PM.grad(Xk , Yk , t_prev).T * (t - t_prev) - (1 / (2 * gamma))*np.linalg.norm(t - t_prev) ** 2
                
            self.t = t
            print("Training finished")
            
        if method == "FISTA" :
            
            for l in range(nmax) :
            
                #Block detection
                k = PM.MOM_rand(PM.P_quadra(X , Y , t) , self.K)[1] 
                Xk = X[k]
                Yk = Y[k]
                #Beginning of backtracking with c = 1/2
                gamma = 1
                t_prev = t
                F = PM.quadra_loss(Xk , Yk , t_prev)
                
                t = PM.soft_thresholding(self.lamb * gamma , z - gamma * PM.grad(Xk , Yk , z))
                delta = PM.quadra_loss(Xk , Yk , t) - F - PM.grad(Xk , Yk , t_prev).T * (t - t_prev) - (1 / (2 * gamma)) * np.linalg.norm(t - t_prev) ** 2
                
                while delta > 1e-3 :
                    
                    gamma *= mu
                    t = PM.soft_thresholding(self.lamb * gamma,z - gamma * PM.grad(Xk , Yk , z))
                    delta = PM.quadra_loss(Xk , Yk , t) - F - PM.grad(Xk , Yk , t_prev).T * (t - t_prev) - (1 / (2 * gamma)) * np.linalg.norm(t - t_prev) ** 2
                    
                z = t + (l / (l + 3)) * (t - t_prev) 
                
            self.t = t
            print("Training finished")
            
        if method == "SUBGRAD" :
             
            for l in range(iter_max) :
       
                k = PM.MOM_rand(PM.P_quadra(X , Y , t) , self.K)[1] 
                Xk = X[k]
                Yk = Y[k]
                   
                t = t - step_size * PM.subgrad(Xk , Yk , t , self.lamb) / np.sqrt(l + 1)  
                   
            self.t = t
            print("Training finished")
    
    def predict(self , X):
        
        return X * self.t
    
    def score(self):
        pass